# Attention Mechanism Demo on Keras: Machine Translation Example (Many-to-Many, encoder-decoder)

In this demo, we will show you how to create a machine translator using Keras. This demo is inspired by Andrew Ng's deeplearning.ai course on sequence models. (Programming Assignment: Neural Machine Translation with Attention)    In this demo, we create a machine translator to translate dates in various formats  into dates in an ISO format. 

In [1]:
%matplotlib inline
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

import random


Using TensorFlow backend.


## Generate Dataset
We generate a toy dataset using datetime library.  A target output only comes in one format (iso format), while there are three different date format for an input.

In [2]:
#Generating a toy dataset
import datetime
base = datetime.datetime.today()
base = datetime.date(base.year, base.month, base.day)
date_list = [base - datetime.timedelta(days=x) for x in range(0, 15000)]

In [3]:
target_date_list = [date.isoformat() for date in date_list] 
print(target_date_list[0])

2018-03-18


In [4]:
from random import randint
random.seed(42)
input_date_list = list()
for date in date_list:
    random_num = randint(0, 2)
    if random_num == 0:
        input_date_list.append(date.strftime("%d/%m/%y"))#"11/03/02"
    elif random_num == 1:
        input_date_list.append(date.strftime("%A %d %B %Y")) #"Monday 11 March 2002"
    elif random_num == 2: 
        input_date_list.append(date.strftime("%d %B %Y")) #"11 March 2002"

In [5]:
for input_sample, target_sample in zip(input_date_list[0:10],target_date_list[0:10]):
    print(input_sample,target_sample)

18 March 2018 2018-03-18
17/03/18 2018-03-17
16/03/18 2018-03-16
15 March 2018 2018-03-15
Wednesday 14 March 2018 2018-03-14
13/03/18 2018-03-13
12/03/18 2018-03-12
11/03/18 2018-03-11
10 March 2018 2018-03-10
09/03/18 2018-03-09


In [6]:
#Preprocessing
input_chars = list(set(''.join(input_date_list)))
output_chars = list(set(''.join(target_date_list)))
data_size, vocab_size = len(input_date_list), len(input_chars)
output_vocab_size = len(output_chars)
print('There are %d lines and %d unique characters in your input data.' % (data_size, vocab_size))
maxlen = len( max(input_date_list, key=len)) #max input length

There are 15000 lines and 41 unique characters in your input data.


In [7]:
print("Max input length:", maxlen)

Max input length: 27


In [8]:
sorted_chars= sorted(input_chars)
sorted_output_chars= sorted(output_chars)
#Input
char_to_ix = { ch:i for i,ch in enumerate(sorted_chars) }
ix_to_char = { i:ch for i,ch in enumerate(sorted_chars) } #reverse dictionary
#Output
output_char_to_ix = { ch:i for i,ch in enumerate(sorted_output_chars) }
ix_to_output_char = { i:ch for i,ch in enumerate(sorted_output_chars) } #reverse dictionary

print(ix_to_char)

{0: ' ', 1: '/', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9', 12: 'A', 13: 'D', 14: 'F', 15: 'J', 16: 'M', 17: 'N', 18: 'O', 19: 'S', 20: 'T', 21: 'W', 22: 'a', 23: 'b', 24: 'c', 25: 'd', 26: 'e', 27: 'g', 28: 'h', 29: 'i', 30: 'l', 31: 'm', 32: 'n', 33: 'o', 34: 'p', 35: 'r', 36: 's', 37: 't', 38: 'u', 39: 'v', 40: 'y'}


In [9]:
m=15000
Tx=maxlen
Ty=10

In [10]:
X = []
for line in input_date_list:
    temp=[]
    for char in line:
        temp.append(char_to_ix[char])
    X.append(temp)
Y = []
for line in target_date_list:
    temp=[]
    for char in line:
        temp.append(output_char_to_ix[char])
    Y.append(temp)    

X = pad_sequences(X,maxlen=maxlen)
Y = pad_sequences(Y,maxlen=10)

X= to_categorical(X,vocab_size)
X=X.reshape(data_size,maxlen ,vocab_size)

Y= to_categorical(Y,output_vocab_size)
Y=Y.reshape(data_size,10 ,output_vocab_size)
print(X.shape,Y.shape)

(15000, 27, 41) (15000, 10, 11)


# Attention Mechanism
![attn_mech](https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW6/attn_mech.png)

In [11]:
#These are global variables (shared layers)
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
#Attention function###
fattn_1 = Dense(10, activation = "tanh")
fattn_2 = Dense(1, activation = "relu")
###
activator = Activation("softmax", name='attention_scores') 
dotor = Dot(axes = 1)

In [12]:
def one_step_attention(a, s_prev):

    # Repeat the decoder hidden state to concat with encoder hidden states
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    # attention function
    e = fattn_1(concat)
    energies =fattn_2(e)
    # calculate attention_scores (softmax)
    attention_scores = activator(energies)
    #calculate a context vector
    context = dotor([attention_scores,a])

    return context

# The model
![rnn_model](https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW6/rnn_date.png)

In [13]:
n_h = 32 #hidden dimensions for encoder 
n_s = 64 #hidden dimensions for decoder
decoder_LSTM_cell = LSTM(n_s, return_state = True) #decoder_LSTM_cell
output_layer = Dense(output_vocab_size, activation="softmax") #softmax output layer

In [14]:
def model(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_h -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    vocab_size -- size of the input vocab
    output_vocab_size -- size of the output vocab

    Returns:
    model -- Keras model instance
    """
    
    # Define the input of your model
    X = Input(shape=(Tx, vocab_size))
    # Define hidden state and cell state for decoder_LSTM_Cell
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = list()

    #Encoder Bi-LSTM
    h = Bidirectional(LSTM(n_h, return_sequences=True),input_shape=(m, Tx, n_h*2))(X)
  
    #Iterate for Ty steps (Decoding)
    for t in range(Ty):
    
        #Perform one step of the attention mechanism to calculate the context vector at timestep t
        context = one_step_attention(h, s)
       
        # Feed the context vector to the decoder LSTM cell
        s, _, c = decoder_LSTM_cell(context,initial_state=[s,c])
           
        # Pass the decoder hidden output to the output layer (softmax)
        out = output_layer(s)
        
        # Append an output list with the current output
        outputs.append(out)
    
    #Create model instance
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    return model

In [15]:
model = model(Tx, Ty, n_h, n_s, vocab_size, output_vocab_size)

In [16]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 27, 41)        0                                            
____________________________________________________________________________________________________
s0 (InputLayer)                  (None, 64)            0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 27, 64)        18944       input_1[0][0]                    
____________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)   (None, 27, 64)        0           s0[0][0]                         
                                                                   lstm_1[0][0]            

In [17]:
opt = Adam(lr= 0.01, decay = 0.01)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [18]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Y.swapaxes(0,1))

In [19]:
model.fit([X, s0, c0], outputs, epochs=20, batch_size=120)

Epoch 1/20
15000/15000 [==============================] - 18s - loss: 12.7681 - dense_3_loss_1: 0.4985 - dense_3_loss_2: 0.5289 - dense_3_loss_3: 1.1394 - dense_3_loss_4: 2.2460 - dense_3_loss_5: 0.8241 - dense_3_loss_6: 1.0804 - dense_3_loss_7: 2.0244 - dense_3_loss_8: 0.8711 - dense_3_loss_9: 1.4234 - dense_3_loss_10: 2.1319 - dense_3_acc_1: 0.8742 - dense_3_acc_2: 0.8511 - dense_3_acc_3: 0.6051 - dense_3_acc_4: 0.2209 - dense_3_acc_5: 0.7788 - dense_3_acc_6: 0.5636 - dense_3_acc_7: 0.2467 - dense_3_acc_8: 0.7443 - dense_3_acc_9: 0.4276 - dense_3_acc_10: 0.2357    
Epoch 2/20
15000/15000 [==============================] - 14s - loss: 4.0360 - dense_3_loss_1: 0.0636 - dense_3_loss_2: 0.0535 - dense_3_loss_3: 0.3745 - dense_3_loss_4: 0.7920 - dense_3_loss_5: 0.0188 - dense_3_loss_6: 0.2373 - dense_3_loss_7: 0.9851 - dense_3_loss_8: 0.0174 - dense_3_loss_9: 0.5399 - dense_3_loss_10: 0.9539 - dense_3_acc_1: 0.9819 - dense_3_acc_2: 0.9817 - dense_3_acc_3: 0.8570 - dense_3_acc_4: 0.7469 - 

# Let's do some "translation"

In [20]:
def prep_input(input_list):
    X = []
    for line in input_list:
        temp=[]
        for char in line:
            temp.append(char_to_ix[char])
        X.append(temp)
    X = pad_sequences(X,maxlen=maxlen)
    X= to_categorical(X,vocab_size)
    X=X.reshape(len(input_list),maxlen ,vocab_size)
    
    return X

EXAMPLES = ['3 May 1999', '05 October 2009', '30 August 2016', '11 July 2000', 'Saturday 19 May 2018', '3 March 2001', '1 March 2001']
EXAMPLES = prep_input(EXAMPLES)
prediction = model.predict([EXAMPLES , s0, c0])
prediction = np.swapaxes(prediction,0,1)
prediction = np.argmax(prediction, axis = -1)

for j in range(len(prediction)):
    output = "".join([ix_to_output_char[int(i)] for i in prediction[j]])
    print(output)

1999-05-33
2009-10-15
2016-08-30
2000-07-11
2018-05-19
2001-03-33
2001-03-10


# Extra: Plot the attention map